In [1]:
# Para no tener que reiniciar el kernel
%load_ext autoreload
%autoreload 2
import sys  # System-specific parameters and functions
sys.path.append("../")  # Adding the parent directory to the system path
from tqdm import tqdm
from src import sp_api as sa
import numpy as np
import pandas as pd
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select

# Para trabajar con Mongo
# ------------------------------------------------------------------------------
import pymongo
import re

In [2]:
cliente = pymongo.MongoClient()
# nos conectamos con una BBDD 
imdb = cliente['IMDB']
peliculas = imdb['Peliculas']
actores = imdb['Actores']

In [12]:
driver = webdriver.Chrome()

driver.maximize_window()

driver.get("https://www.imdb.com/")

for actor in tqdm(list(actores.find())):
    if  type(actor['conocido']) is float:
        continue
    elif len(actor['conocido'])<1:
        continue

    elif type(actor['conocido'][0]) is float:
        continue
    elif len(actor['conocido'][0]) == 9:
        continue


    try:
        busqueda_actor = driver.find_element('css selector', '#suggestion-search').send_keys(actor['nombre'], Keys.ENTER)
        driver.implicitly_wait(3)
        driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-9b6c53e1-0.jTQUPo > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
                                             
    except:
        driver.close()
        #Abrimos el Chrome
        driver = webdriver.Chrome()

        # Maximizar la ventana del navegador
        driver.maximize_window()

        # Navegar a la página web de IMDB 
        driver.get("https://www.imdb.com")
        busqueda_actor = driver.find_element('css selector', '#suggestion-search').send_keys(actor['nombre'], Keys.ENTER)
        try:
            driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-9b6c53e1-0.jTQUPo > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
        except:
            continue


    lista_conocidos = []
    try:
        for i in range(1,5):
            nuevo_conocido = driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]/div/div[2]/div[{i}]/div[2]/div[1]/a').get_attribute('href').split('/')[5]
            lista_conocidos.append(nuevo_conocido)                                         
    except:
        pass

    actores.update_one({'nombre':actor['nombre']},{'$set': {'conocido': lista_conocidos}})


        



driver.close()


 49%|████▉     | 6320/12873 [36:48<38:10,  2.86it/s]   


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.234)
Stacktrace:
0   chromedriver                        0x00000001049ac4dc chromedriver + 4162780
1   chromedriver                        0x00000001049a4664 chromedriver + 4130404
2   chromedriver                        0x00000001045fbbc0 chromedriver + 293824
3   chromedriver                        0x00000001045d46c0 chromedriver + 132800
4   chromedriver                        0x000000010466d25c chromedriver + 758364
5   chromedriver                        0x0000000104672b70 chromedriver + 781168
6   chromedriver                        0x00000001046356bc chromedriver + 530108
7   chromedriver                        0x0000000104636930 chromedriver + 534832
8   chromedriver                        0x0000000104971e08 chromedriver + 3923464
9   chromedriver                        0x00000001049763dc chromedriver + 3941340
10  chromedriver  

In [57]:
driver = webdriver.Chrome()

driver.maximize_window()

driver.get("https://www.imdb.com/")

for pelicula in tqdm(list(peliculas.find())):
    for actor in pelicula['actores']:
        if actores.find_one({'nombre': actor}):
            continue
        dicc = {}
        dicc['nombre'] = actor
        #actores.update_one({'nombre': actor},{'$addToSet': {'peliculas': pelicula['id_IMDB']}})
        try:
            busqueda_actor = driver.find_element('css selector', '#suggestion-search').send_keys(actor, Keys.ENTER)
            driver.implicitly_wait(3)
            driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-9b6c53e1-0.jTQUPo > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
    
        except:
            driver.close()
            #Abrimos el Chrome
            driver = webdriver.Chrome()

            # Maximizar la ventana del navegador
            driver.maximize_window()

            # Navegar a la página web de IMDB 
            driver.get("https://www.imdb.com")
            busqueda_actor = driver.find_element('css selector', '#suggestion-search').send_keys(actor, Keys.ENTER)
            try:
                driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-9b6c53e1-0.jTQUPo > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-b03627f1-2.gWHDBT > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
            except:
                lista_conocidos = []
                dicc['year'] = np.nan
                dicc['conocido']= lista_conocidos
                dicc['roles']= np.nan
                dicc['premios'] = np.nan
                dicc['nominaciones'] = np.nan
                actores.insert_one(dicc)
                continue

        try: 
            dicc['year'] = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-75c84411-0.icfMdl > div > section > div > div.sc-aa354f15-1.cgHAsh.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(22) > div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(4) > div > ul > li.ipc-inline-list__item.test-class-react > a").text.split(",")[-1] 
            dicc['year'] = int(dicc['year'].split()[-1])
                                                    #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.dEqUUl > div.sc-e226b0e3-6.CUzkx > div.sc-e226b0e3-11.kkLqLt > section > aside > div > span:nth-child(2)
        except:
            dicc['year'] = np.nan
        try:
            lista_conocidos = []
            for i in range(1,5):
                conocido = driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]/div/div[2]/div[{i}]/div[2]/div[1]/a').get_attribute('href').split('/')[5]
                                                        
                lista_conocidos.append(conocido)
            dicc['conocido'] =lista_conocidos                   #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid
        except:
            dicc['conocido']= lista_conocidos
        #Esto hay que cambiarlo para que saque todas las cosas que pone, pero como un único str
        try:
            dicc['roles'] = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-75c84411-0.icfMdl > section > div:nth-child(5) > section > section > div.sc-9a2a0028-3.bwWOiy > div > ul").text.split("\n")
        except:
            dicc['roles']= np.nan
        #Faltan premios (solo si tiene oscar)
        try:
            premios = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-75c84411-0.icfMdl > div > section > div > div.sc-aa354f15-1.cgHAsh.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > div > ul').text
            try:
                dicc['premios'] = int(premios.split(' premio')[0])
                nominaci_y =premios.split(' premio')[-1].split(' nominaci')[0]
                dicc['nominaciones'] = int(nominaci_y.split(' y ')[-1])
            except:
                if 'nominaci' in premios: 
                    dicc['nominaciones'] = int(premios.split(' nominaci')[0])
                    dicc['premios'] = np.nan
                else:
                    dicc['premios'] = premios.split(' premio')[0]
                    dicc['nominaciones'] = np.nan





        except:
            dicc['premios'] = np.nan
            dicc['nominaciones'] = np.nan



        
        
        actores.insert_one(dicc)
driver.close()

 14%|█▍        | 2805/20237 [11:30:58<71:34:11, 14.78s/it] 


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x000000010075c4dc chromedriver + 4162780
1   chromedriver                        0x0000000100754664 chromedriver + 4130404
2   chromedriver                        0x00000001003ab9f8 chromedriver + 293368
3   chromedriver                        0x00000001003e55ec chromedriver + 529900
4   chromedriver                        0x00000001003e6930 chromedriver + 534832
5   chromedriver                        0x0000000100721e08 chromedriver + 3923464
6   chromedriver                        0x00000001007263dc chromedriver + 3941340
7   chromedriver                        0x000000010070a038 chromedriver + 3825720
8   chromedriver                        0x0000000100726f3c chromedriver + 3944252
9   chromedriver                        0x00000001006fc6f4 chromedriver + 3770100
10  chromedriver                        0x0000000100743980 chromedriver + 4061568
11  chromedriver                  

In [53]:
actores_eliminar = list(actores.find({"$and":[{'year':float('nan')}, {'roles':float('nan')}, {'premios':float('nan')}]}))

In [46]:
driver = webdriver.Chrome()

driver.maximize_window()

driver.get("https://www.imdb.com/es-es/name/nm0000151/?ref_=nv_sr_srsg_0_tt_0_nm_8_in_0_q_Morgan%2520f")

dicc = {}

try: 
    dicc['year'] = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-75c84411-0.icfMdl > div > section > div > div.sc-aa354f15-1.cgHAsh.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(22) > div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(4) > div > ul > li.ipc-inline-list__item.test-class-react > a").text.split(",")[-1] 
    dicc['year'] = int(dicc['year'].split()[-1])
                                            #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.dEqUUl > div.sc-e226b0e3-6.CUzkx > div.sc-e226b0e3-11.kkLqLt > section > aside > div > span:nth-child(2)
except:
    dicc['year'] = np.nan
try:
    lista_conocidos = []
    for i in range(1,5):
        conocido = driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]/div/div[2]/div[{i}]/div[2]/div[1]/a').get_attribute('href').split('/')[5]
                                                  
        lista_conocidos.append(conocido)
    dicc['conocido'] =lista_conocidos                   #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid
except:
    dicc['conocido']= lista_conocidos
#Esto hay que cambiarlo para que saque todas las cosas que pone, pero como un único str
try:
    dicc['roles'] = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-75c84411-0.icfMdl > section > div:nth-child(5) > section > section > div.sc-9a2a0028-3.bwWOiy > div > ul").text.split("\n")
except:
    dicc['roles']= np.nan
#Faltan premios (solo si tiene oscar)
try:
    premios = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-75c84411-0.icfMdl > div > section > div > div.sc-aa354f15-1.cgHAsh.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > div > ul').text
    try:
        dicc['premios'] = int(premios.split(' premio')[0])
        nominaci_y =premios.split(' premio')[-1].split(' nominaci')[0]
        dicc['nominaciones'] = int(nominaci_y.split(' y ')[-1])
    except:
        if 'nominaci' in premios: 
            dicc['nominaciones'] = int(premios.split(' nominaci')[0])
            dicc['premios'] = np.nan
        else:
            dicc['premios'] = premios.split(' premio')[0]
            dicc['nominaciones'] = np.nan





except:
    dicc['premios'] = np.nan
    dicc['nominaciones'] = np.nan


driver.close()

In [47]:
dicc

{'year': 1937,
 'conocido': ['tt0114369', 'tt0097239', 'tt0405159', 'tt0468569'],
 'roles': ['Reparto', 'Producción', 'Dirección'],
 'premios': 66,
 'nominaciones': 88}

In [55]:
for actor in actores_eliminar:
    actores.delete_one({"_id": actor['_id']})

In [56]:
actores_eliminar

[{'_id': ObjectId('679f6195f607b720a11d8ae0'),
  'nombre': 'Morgan Freeman',
  'year': nan,
  'conocido': [],
  'roles': nan,
  'premios': nan,
  'nominaciones': nan},
 {'_id': ObjectId('679f61a3f607b720a11d8ae1'),
  'nombre': 'Brian Bosworth',
  'year': nan,
  'conocido': [],
  'roles': nan,
  'premios': nan,
  'nominaciones': nan},
 {'_id': ObjectId('679f61c0f607b720a11d8ae2'),
  'nombre': 'Wil Wheaton',
  'year': nan,
  'conocido': [],
  'roles': nan,
  'premios': nan,
  'nominaciones': nan},
 {'_id': ObjectId('679f61dcf607b720a11d8ae3'),
  'nombre': 'Philip Dorr',
  'year': nan,
  'conocido': [],
  'roles': nan,
  'premios': nan,
  'nominaciones': nan},
 {'_id': ObjectId('679f61e6f607b720a11d8ae4'),
  'nombre': 'Deepak Shirke',
  'year': nan,
  'conocido': [],
  'roles': nan,
  'premios': nan,
  'nominaciones': nan},
 {'_id': ObjectId('679f61f0f607b720a11d8ae5'),
  'nombre': 'Kader Khan',
  'year': nan,
  'conocido': [],
  'roles': nan,
  'premios': nan,
  'nominaciones': nan},
 {'